
# ЛАБОРАТОРНА РОБОТА  

## "Лінійні та нелінійні регресійні моделі МН"

__Метою__ лабораторної роботи є набуття практичних навичок використання модулів бібліотеки `Scikit-learn` для вирішення наступних задач:

- визначення суттєвих показчиків для регресійної моделі
- пошук та настроювання гіперпараметрів лінійних та нелінійних регресійних моделей

__Результатом__ виконання лабораторної роботи є серія моделей які прогнозують ціну кватрир на вторинному ринку житла

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('apartment_transformed.csv')
data.shape

(376, 176)

In [2]:
# відокремити ціловий показчик 'price'
price = data.Price

# зилишити в 'data' тільки незалежні показчики
data = data.drop(['Price'], axis=1)

data.shape, price.shape    # ((676, 301), (676,))

((376, 175), (376,))

## 1. Пошук значущих ознак

### Теоретичне введення

__Значущі (суттеві) ознаки__ це дані, які мають сильну кореляцію або вплив на результат або прогноз моделі.

Ці позазчики визначаються за допомогою процесу, який називається __відбором ознак__ ([Feature selection](https://en.wikipedia.org/wiki/Feature_selection)), який передбачає _оцінку_ та _ранжування важливості_ різних змінних у наборі даних. Це можна зробити за дпомогою статистичних тестів, кореляційного аналізу або алгоритмів машинного навчання.

Після визначення значущих ознак їх можна використовувати для навчання моделі машинного навчання.

Це може бути особливо важливим у моделях, де дані складні та містять багато змінних. Тому ідентифікація  суттєвих ознак може допомогти зменшити розмірність даних і покращити продуктивність моделі.

Класи в модулі [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html) можна використовувати для вибору функцій/зменшення розмірності на вибіркових наборах або для покращення показників точності оцінювачів, або для підвищення їх продуктивності на масивах даних з дуже великою розмірністю.

### Завдання

Відібрати з вхідного набору `data` 7 найбільш суттєвих показчиків для регрісійної моделі машинного навчання.

In [3]:
# імпортувати з модуля 'feature_selection' селектор ознак 'SelectKBest' 
# та регрісійний тест 'f_regression'
from sklearn.feature_selection import SelectKBest, f_regression

In [4]:
# побудувати селектор 7 ознак на f-регресорі
kbest_selector = SelectKBest(f_regression, k=7)

# застосувати селектор для побудови списку ознак
data_selected = kbest_selector.fit_transform(data, price)

In [5]:
# зберегти імена визначених селектором найбільш суттєвих ознак
best_features = kbest_selector.get_feature_names_out()

In [6]:
# побудувати датафрейм на визначених ознаках
data = pd.DataFrame(data=data_selected)
data.columns = best_features
data.head()

,ohe_categirical_encoder__street_Мічуріна,ohe_categirical_encoder__street_Саксаганського,numeric_encoder__rooms,numeric_encoder__price_per_m2,numeric_encoder__area_total,numeric_encoder__area_living,numeric_encoder__area_comfort
0,0.0,0.0,-1.135681,-0.139714,-1.050589,-0.997549,-0.431288
1,0.0,0.0,-0.176094,-0.117996,-0.297370,-0.152619,-0.321841
2,0.0,0.0,-0.176094,-0.019208,-0.276447,-0.575084,0.006501
3,0.0,0.0,-0.176094,-0.118989,-0.255525,-0.152619,-0.394806
4,0.0,0.0,-1.135681,-0.095409,-0.778594,-0.152619,-0.796113


### Висновки

_описати загальну статистичну характеристику отриманого датасети та зробити висновки щодо можливості його використання для подальшого аналізу_

Отриманий датасет містить 7 найбільш суттєвих колонок(ознак). Такий датасет дозволить запобігти перенавчанню моделей, що могло статися при використанні усіх ознак з початкового датасету.

## 2. Множинна лінійна регресія

### Теоретичне введення

__Множинна лінійна регресія__ — це статистичний метод, який використовується для встановлення зв’язку між _залежною_ (цільвою) змінною $\textbf y$ та _кількома_ незалежними змінними $\textbf [X]$.  

__Метою__ множинної лінійної регресії є знаходження найкращого лінійного зв’язку між залежною змінною та незалежними змінними, який виражається у вигляді рівняння:
$$y = b_0 + b_1 x_1 + b_2 x_2 + ... + b_n x_n$$

Найкращій зв'язок забезпечується знаходженням таких коєфіцієнтів $[B]$, що додають мінімум обраній метриці (MSE, MAE, ...)

### Завдання


Порахувати показчики якості моделі [лінйной множинної регресії](https://uk.mcfairbanks.com/719-multiple-regression-formula) на визначениx п.1 значущих ознаках датасету застосувавши [кросс-валідацію з __10__ сплітами](https://scikit-learn.org/stable/modules/cross_validation.html).

In [7]:
# імпортувати та побудувати лінійний регресор з параметрами за замовчанням
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [8]:
# імпортувати крос-валідатор 'cross_validate' з модуля 'model_selection'
from sklearn.model_selection import cross_validate, KFold

In [9]:
# отримати результати крос-валідації по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_mul = cross_validate(lr, data, price,
                                scoring=['neg_mean_absolute_percentage_error', 'r2'],
                                cv=KFold(n_splits=10, shuffle=True), return_train_score=True)

#### занести результати в датафрейм 'cv_results_mul ' наступного вигляду:

------------

|помилка тесту в %   |коєф. R2 тесту  | помилка навчання в %  | коєф. R2 навчання  |
| :------------:|:------------:|:------------:|:------------:|
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  ... | ...  | ...  | ...  |


In [10]:
cv_results_mul = pd.DataFrame({
    'помилка тесту в %': cv_results_mul['test_neg_mean_absolute_percentage_error'],
    'коеф. R2 тесту': cv_results_mul['test_r2'],
    'помилка навчання в %': cv_results_mul['train_neg_mean_absolute_percentage_error'],
    'коеф. R2 навчання': cv_results_mul['train_r2']
})
cv_results_mul.head()

,помилка тесту в %,коеф. R2 тесту,помилка навчання в %,коеф. R2 навчання
0,-0.381270,0.677765,-0.304725,0.994666
1,-0.500728,0.579671,-0.442997,0.988139
2,-0.371439,0.769507,-0.465052,0.986183
3,-0.444636,0.856462,-0.431125,0.986634
4,-0.459122,0.551667,-0.462941,0.986782


In [11]:
# продовжити наступні команди виводу:
print ("середня помилка навчання = ", np.mean(cv_results_mul['помилка навчання в %']))
print ("середня помилка тесту = ", np.mean(cv_results_mul['помилка тесту в %']))

середня помилка навчання =  -0.43430899431877484
середня помилка тесту =  -0.44193916015487644


### Висновки

_зпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._

## 3. Гребнева (Ridge) регресія

### Теоретичне введення

__Гребнева регресія__ — це техніка регулярізації, яка використовується в машинному навчанні для запобігання перенавчанню лінійних регресійних моделей за рахунок додавання штрафу до функції втрат регресійної моделі, яка зменшує величину коефіцієнтів до нуля.

$$\min_w\sum_{i=1}^n(y_i - x_i^w)^2 + \lambda|w|_2^2$$

Розмір штрафу визначається [нормою L2](https://craftappmobile.com/l1-vs-l2-regularization/) вектора коефіцієнтів, помноженою на гіперпараметр $\large \lambda$.




### Завдання

Побудувати модель на основі `ridge-регресії` та за допомогою [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) знайти таке значення _L2-регулярізатора_, яке буде
мінізувати обрані метрики якості моделі.
Для побудови моделі скоистатися датасетом, що отримано в  __завданні 1__ лабораторної роботи


In [12]:
# імпортувати ridge-регресор з модуля `sklearn.linear_model`
from sklearn.linear_model import Ridge

# побудувати регресор
ridge = Ridge(random_state=2, fit_intercept=False)

In [13]:
# імпортувати сітку пошуку `GridSearchCV` з модулю sklearn.model_selection
from sklearn.model_selection import GridSearchCV

# визначити параметр равномірного пошуку 100 значень параметеру `alpha` в диапазоні 0-100000 
grid_params = {
    'alpha': np.arange(0, 100000, 1000),
    'max_iter': [1500, 3000],
}

In [14]:
%%time

# побудувати та натренувати гребневу регресійну модель на сітці 'grid_params'
# в якості критерія оцінки якості взяти метрику `neg_mean_absolute_percentage_error`

# створюємо сітку пошуку та тренуємо на ній модель
grid_search_model = GridSearchCV(ridge, param_grid=grid_params, cv=10,
                                 scoring='neg_mean_absolute_percentage_error')

grid_search_model.fit(data, price)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.3 s


GridSearchCV(cv=10, estimator=Ridge(fit_intercept=False, random_state=2),
             param_grid={'alpha': array([    0,  1000,  2000,  3000,  4000,  5000,  6000,  7000,  8000,
        9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000,
       18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000,
       27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000,
       36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 4...
       45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000,
       54000, 55000, 56000, 57000, 58000, 59000, 60000, 61000, 62000,
       63000, 64000, 65000, 66000, 67000, 68000, 69000, 70000, 71000,
       72000, 73000, 74000, 75000, 76000, 77000, 78000, 79000, 80000,
       81000, 82000, 83000, 84000, 85000, 86000, 87000, 88000, 89000,
       90000, 91000, 92000, 93000, 94000, 95000, 96000, 97000, 98000,
       99000]),
                         'max_iter': [1500, 3000]},
             scoring='neg_mean_absolute_percentage_error')

In [15]:
# вивести найкращій естіматор (best_estimator_), та найкраще значення обраної метрики (best_score_)
grid_search_model.best_estimator_, grid_search_model.best_score_

(Ridge(alpha=99000, fit_intercept=False, max_iter=1500, random_state=2),
 -1.011978280681753)

### Висновки

_cпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._

## 3. Поліноміальна регресія

### Теоретичне введення

__Поліноміальна регресія__ — це тип нелінійної регресії, у якому зв’язок між незалежною змінною $\large x$ і залежною змінною $\large y$ моделюється як поліноміальна функція n-го ступеня:

$$ y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + \cdots + \beta_n x^n + \varepsilon $$

__Метою__ поліноміальної регресії є знаходження значень коефіцієнтів $ β_i$, які найкраще відповідають даним.

### Завдання


Порахувати показчики якості моделі на [поліноміальній регресії](https://uk.wikipedia.org/wiki/Поліноміальна_регресія) на визначених в п.1 значущих ознаках датасету, попередньо розширивши датасет за допомогою трансформера [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [16]:
# імпортувати модуль preprocessing.PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures

# побудувати трансформер ступеня 2 для побудови додаткових ознак в датасеті
poly = PolynomialFeatures(degree=2)
poly_df = poly.fit_transform(data, price)

In [17]:
# визначити імена відібраних показчиків
poly_features_names = poly.get_feature_names_out()

In [18]:
# побудувати датасет на визначеному поліномі `poly`
data_poly = pd.DataFrame(poly_df)
data_poly.columns=poly_features_names
data_poly.shape

(376, 36)

In [19]:
# отримати результати крос-валідації на множинном регресорі `lr` по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_poly = cross_validate(lr, data_poly, price,
                                 scoring=['neg_mean_absolute_percentage_error', 'r2'],
                                 cv=KFold(n_splits=10, shuffle=True), return_train_score=True)

In [20]:
# занести результати крос-валідації: помилка тесту, помилка навчання та відповідні коефіцієнти 
# детермінаційї в датафрейм `cv_results_poly`. 
cv_results_poly = cv_results_mul = pd.DataFrame({
    'помилка тесту в %': cv_results_poly['test_neg_mean_absolute_percentage_error'],
    'коеф. R2 тесту': cv_results_poly['test_r2'],
    'помилка навчання в %': cv_results_poly['train_neg_mean_absolute_percentage_error'],
    'коеф. R2 навчання': cv_results_poly['train_r2']
})

cv_results_poly.head()

,помилка тесту в %,коеф. R2 тесту,помилка навчання в %,коеф. R2 навчання
0,-1.201199e-01,6.218861e-01,-0.077170,0.999547
1,-1.266887e-01,9.877676e-01,-0.081267,0.997612
2,-9.534936e-02,9.600573e-01,-0.092540,0.997490
3,-5.862347e-02,9.921817e-01,-0.091148,0.997480
4,-7.341466e+08,-2.768136e+20,-0.087972,0.962104


In [ ]:
# за допомогою крос-валідатора 'cross_val_predict' побудувати прогноз 'price_pred' 
# на лінійному регресорі на 10 сплітах
from sklearn.model_selection import cross_val_predict

price_pred = cross_val_predict(lr, data_poly, price, cv=10)

<class 'NameError'>: name 'lr' is not defined

In [2]:
# вивести порівняльну таблицю з двох колонок: ціна реальна, ціна прогнозна
pred = pd.DataFrame({'ціна реальна': price,
                     'ціна прогнозна': np.round(price_pred)})
pred.head(5)

<class 'NameError'>: name 'pd' is not defined

In [ ]:
# натренувати регресор `lr` на поліноміальних ознаках `data_poly`
lr.fit(data_poly, price)

In [ ]:
# сформувати таблицю коєфіцієнтів поліному
coef = pd.DataFrame({'Ознаки': poly_features_names,
                     'коеф.регресора': lr.coef_.astype('int')})

### Висновки

_Базуючись на значенях метрик абсолютної помилки та r2-оцінки, сформулювати вашу думку чи відповідає поліноміальна модель вимогам якості та дати характеристику декільком коефіцієнтам (3-4) на свій вибір._ 

...

## 5. Зберігання побудованх моделей

In [1]:
# зберегти лінийну, гребневу та поліноміальну моделі у відпрвідних pickle-файлах:
# 'lin_model.pkl', 'ridge_model.pkl', 'poly_model.pkl'

import pickle

model_names = ['lin_model.pkl', 'ridge_model.pkl', 'poly_model.pkl']
models = [lr, ridge, poly]

for i in range(len(model_names)):
    with open(model_names[i], 'wb') as f:
        pickle.dump(models[i], f)

<class 'NameError'>: name 'lr' is not defined